In [1]:
%load_ext sql

In [2]:
%sql mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?trusted_connection=yes&driver=SQL+Server

### 1. Report the all the loan contracts which just have been signed in January 2022, and all the loan contracts of which installment should be collected in January 2022.
• The result should provide a table report with data of contract numbers, signed dates, installment paid dates, loan amounts, and data of clients related to the contracts including full names, date of birth, and addresses

### Answer
- As the data's latest signing date is on 9/7/2013 and the main purpose of this test is to showcase my SQL abilities, I will instead query all the loan contracts which were signed in January, 2013 and all the contracts of which installments were to be collected in January, 2013

In [3]:
# Select the latest signed contract's date in the data
%sql SELECT MAX(DATE_SIGNATURE_CONTRACT) as max_signing_date FROM dbo.DC_CONTRACT    

 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


max_signing_date
2013-09-07


In [4]:
%%sql 
SELECT CO.TEXT_CONTRACT_NUMBER AS CONTRACT_NUMBER
       ,MIN(CAST(CO.DATE_SIGNATURE_CONTRACT AS DATE)) AS SIGNED_DATE
       ,MAX(CAST(CO.DATE_LAST_PAYMENT AS DATE)) AS INSTALLMENT_PAID_DATES
       ,SUM(F.AMT_CREDIT_LOAN) AS LOAN_AMOUNT
       ,MAX(CL.NAME_CLIENT) AS CLIENT_FIRST_NAME
       ,MAX(CAST(CL.DATE_BIRTH AS DATE)) AS CLIENT_BIRTHDATE
       ,COALESCE(MAX(A.NAME_STREET) + 'St., ', '') 
       + COALESCE('Ward ' + MAX(A.NAME_WARD) + ', ', '') 
       + COALESCE(MAX(A.NAME_DISTICT) + ', ', '') 
       + COALESCE(MAX(A.NAME_PROVINCE) + ', ', '') 
       + COALESCE(MAX(A.NAME_COUNTRY),'') AS CLIENT_ADDRESS
FROM DC_CONTRACT CO JOIN F_CONTRACT_BASE_AD F ON CO.SKP_CONTRACT = F.SKP_CONTRACT
                    JOIN DC_CLIENT CL ON F.SKP_CLIENT = CL.SKP_CLIENT
                    JOIN DC_ADDRESS A ON CL.SKP_ADDRESS = A.SKP_ADDERSS
WHERE CAST(CO.DATE_SIGNATURE_CONTRACT AS DATE) >= '2013-01-01' AND CAST(CO.DATE_LAST_PAYMENT AS DATE) >= '2013-01-01' 
GROUP BY TEXT_CONTRACT_NUMBER


 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


CONTRACT_NUMBER,SIGNED_DATE,INSTALLMENT_PAID_DATES,LOAN_AMOUNT,CLIENT_FIRST_NAME,CLIENT_BIRTHDATE,CLIENT_ADDRESS
3140005704,2013-09-01,2014-03-01,3432000.0,MINH,1991-10-11,"Nam Ki Khoi Nghia St., Ward 18, 10, TP.HCM, VIET NAM"
3140012036,2013-09-01,2014-02-01,7693000.0,LE,1985-01-01,"Le Lai St., Ward 11, VUNG TAU, BA RIA VUNG TAU, VIET NAM"


2. Report the sales performance (sales & profits) of whole year 2021 of each product and each 
region: Hanoi, Ho Chi Minh City, Da Nang City, and Others (the rest of the provinces) in absolute 
numbers and in percentage shares

### Answer
With my own knowledge, as 

In [15]:
%%sql
WITH CTE_F AS (
    SELECT
        P.PRODUCT_CODE
       ,P.PRODUCT_NAME
       ,CASE WHEN A.NAME_PROVINCE IN ('TP.HCM') THEN 'Ho Chi Minh City'
             WHEN A.NAME_PROVINCE IN ('DA NANG') THEN 'Da Nang City'
             WHEN A.NAME_PROVINCE IN ('HA NOI') THEN 'Hanoi'
             ELSE ('Others') 
        END AS REGION
        ,(F.AMT_DOWN_PAYMENT - F.AMT_PRINCIPAL + F.AMT_INTEREST - F.AMT_PROVISION - F.AMT_INSURANCE) AS PROFIT
        ,(F.AMT_DOWN_PAYMENT + F.AMT_INTEREST) AS SALES
    FROM DC_PRODUCT P JOIN CTE_F F ON P.SKP_PRODUCT = F.SKP_PRODUCT
                  JOIN DC_SALEROOMS S ON F.SKP_SALESROOM = S.SKP_SALESROOM
                  JOIN DC_ADDRESS D ON S.SKP_ADDERSS = A.SKP_ADDERSS
)

SELECT *
FROM CTE_F

 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
(pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Recursive common table expression 'CTE_F' does not contain a top-level UNION ALL operator. (252) (SQLExecDirectW)")
[SQL: WITH CTE_F AS (
    SELECT
        P.PRODUCT_CODE
       ,P.PRODUCT_NAME
     
    FROM DC_PRODUCT P JOIN CTE_F F ON P.SKP_PRODUCT = F.SKP_PRODUCT
                  JOIN DC_SALEROOMS S ON F.SKP_SALESROOM = S.SKP_SALESROOM
                  JOIN DC_ADDRESS D ON S.SKP_ADDERSS = A.SKP_ADDERSS
)

SELECT *
FROM CTE_F]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [ ]:
PRODUCT_CODE
       ,PRODUCT_NAME
       ,REGION
       ,SUM(F.SALES) AS SALES
       ,SUM(F.PROFIT) AS PROFIT
       ,AVG(F.SALES) OVER(PARTITION BY REGION) AS PERCENTAGE_SALES
       ,AVG(F.PROFIT) OVER(PARTITION BY REGION) AS PERCENTAGE_PROFIT

In [24]:
%%sql
WITH CTE_F AS (
      SELECT 
            *
            ,(AMT_DOWN_PAYMENT - AMT_PRINCIPAL + AMT_INTEREST - AMT_PROVISION - AMT_INSURANCE) AS PROFIT
            ,(AMT_DOWN_PAYMENT + AMT_INTEREST) AS SALES
      FROM  F_CONTRACT_BASE_AD 
)
,CTE_A AS (
      SELECT 
            *
            ,CASE WHEN NAME_PROVINCE IN ('TP.HCM') THEN 'Ho Chi Minh City'
             WHEN NAME_PROVINCE IN ('DA NANG') THEN 'Da Nang City'
             WHEN NAME_PROVINCE IN ('HA NOI') THEN 'Hanoi'
             ELSE ('Others') 
        END AS REGION
      FROM DC_ADDRESS
)


SELECT P.PRODUCT_CODE
       ,P.PRODUCT_NAME
       ,A.REGION
       ,SUM(F.SALES) AS SALES
       ,SUM(F.PROFIT) AS PROFIT
       ,ROUND(SUM(F.SALES) / (SUM(SUM(F.SALES)) OVER(PARTITION BY REGION)), 2) AS PERCENTAGE_SALES
FROM DC_PRODUCT P JOIN CTE_F F ON P.SKP_PRODUCT = F.SKP_PRODUCT
                  JOIN DC_SALEROOMS S ON F.SKP_SALESROOM = S.SKP_SALESROOM
                  JOIN CTE_A A ON S.SKP_ADDERSS = A.SKP_ADDERSS

GROUP BY P.PRODUCT_CODE, P.PRODUCT_NAME, A.REGION



 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


PRODUCT_CODE,PRODUCT_NAME,REGION,SALES,PROFIT,PERCENTAGE_SALES
CD162,VIVO 5-10M,Hanoi,7470000.0,-7055000.0,1.0
CD162,VIVO 5-10M,Ho Chi Minh City,11530000.0,-6644000.0,0.26
CDS37,MW APPLE 8,Ho Chi Minh City,1287000.0,-965000.0,0.03
CDS40,MW DIEN MA,Ho Chi Minh City,30022316.0,-2178368.0,0.67
CDS41,FPT OPPO A,Ho Chi Minh City,1697816.0,-1460368.0,0.04


### 3. Report the numbers of old customers and new customers in each of four regions having newly signed contracts in Quarter 4 of 2021.
• Get a list of top 100 customers who are currently having active contracts ranked based on numbers of active contracts and total loan amounts.

• Assume that old customers are those who have signed at least one contract with the Home Credit VN before Quarter 4 of 2021, and new customers are those who have never had a contract with the Company before Quarter 4 of 2021. 

• Also assume that NAME_CONTRACT_STATUS contains Approved, Signed, Cancelled, and Finished.

In [7]:
%%sql
WITH CTE_CUSTOMER AS
    (
      SELECT CL.SKP_CLIENT, C.DATE_SIGNATURE_CONTRACT, A.NAME_PROVINCE 
      FROM DC_ADDRESS A JOIN DC_CLIENT CL ON A.SKP_ADDERSS = CL.SKP_ADDRESS 
                        JOIN F_CONTRACT_BASE_AD F ON CL.SKP_CLIENT = F.SKP_CLIENT 
                        JOIN DC_CONTRACT C ON F.SKP_CONTRACT = C.SKP_CONTRACT
    )    
SELECT 
    CASE WHEN NAME_PROVINCE IN ('TP.HCM') THEN 'Ho Chi Minh City'
             WHEN NAME_PROVINCE IN ('DA NANG') THEN 'Da Nang City'
             WHEN NAME_PROVINCE IN ('HA NOI') THEN 'Hanoi'
             ELSE 'Others' 
    END AS REGION
    ,COUNT(DISTINCT CASE WHEN CAST(DATE_SIGNATURE_CONTRACT AS DATE) >= '2013-10-01' THEN SKP_CLIENT END) AS TOTAL_NEW_CUSTOMERS
    ,COUNT(DISTINCT CASE WHEN CAST(DATE_SIGNATURE_CONTRACT AS DATE) < '2013-10-01' THEN SKP_CLIENT END) AS TOTAL_OLD_CUSTOMERS
FROM CTE_CUSTOMER 
GROUP BY NAME_PROVINCE
    

 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


REGION,TOTAL_NEW_CUSTOMERS,TOTAL_OLD_CUSTOMERS
Others,0,1
Ho Chi Minh City,0,1


In [8]:
%%sql 
SELECT TOP 100 CL.SKP_CLIENT
               ,MAX(CL.NAME_CLIENT) AS CLIENT_FIRST_NAME
               ,COUNT(C.SKP_CONTRACT) AS NUMBER_OF_CONTRACTS
               ,SUM(F.AMT_CREDIT_LOAN) AS LOAN_AMOUNT
FROM DC_CLIENT CL JOIN F_CONTRACT_BASE_AD F ON CL.SKP_CLIENT = F.SKP_CLIENT
                 JOIN DC_CONTRACT C ON F.SKP_CONTRACT = C.SKP_CONTRACT
                 JOIN CL_CONTRACT_STATUS S ON C.SKP_CONTRACT_STATUS = S.SKP_CREDIT_STATUS
WHERE S.NAME_CREDIT_STATUS = 'Active'
GROUP BY CL.SKP_CLIENT
ORDER BY NUMBER_OF_CONTRACTS DESC, LOAN_AMOUNT DESC

 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


SKP_CLIENT,CLIENT_FIRST_NAME,NUMBER_OF_CONTRACTS,LOAN_AMOUNT
502422,MINH,4,13728000.0
3200785,HOA,1,30000000.0
3331518,THAO,1,24572000.0
2745335,LE,1,7693000.0
5886137,TAM,1,6293000.0
2793058,MINH,1,5694000.0
4681271,DUNG,1,3773000.0
192076,TOAN,1,3493000.0
3915968,SON,1,3380000.0


4. Categorized list of old customers into age segmentation: 18 – 23, 24 – 35, 36 – 50, and > 50

In [9]:
%%sql
WITH CTE_CUSTOMER AS
    (
      SELECT CL.SKP_CLIENT
            ,MAX(CL.NAME_CLIENT) AS NAME_CLIENT
            ,DATEDIFF(year, MIN(CL.DATE_BIRTH), GETDATE()) AS AGE
            ,MIN(C.DATE_SIGNATURE_CONTRACT) AS DATE_SIGNATURE_CONTRACT
      FROM DC_ADDRESS A JOIN DC_CLIENT CL ON A.SKP_ADDERSS = CL.SKP_ADDRESS 
                        JOIN F_CONTRACT_BASE_AD F ON CL.SKP_CLIENT = F.SKP_CLIENT 
                        JOIN DC_CONTRACT C ON F.SKP_CONTRACT = C.SKP_CONTRACT
      GROUP BY CL.SKP_CLIENT
    )    

SELECT SKP_CLIENT
       ,NAME_CLIENT
       ,CASE WHEN AGE BETWEEN 18 AND 23 THEN '18-23'
             WHEN AGE BETWEEN 24 AND 35 THEN '24-35'
             WHEN AGE BETWEEN 36 AND 50 THEN '36-50'
             WHEN AGE > 50 THEN '>50'
       END AS AGE_SEGMENTATION
FROM CTE_CUSTOMER


 * mssql+pyodbc://root@DESKTOP-G1H2LMS/HOMECREDIT?driver=SQL+Server&trusted_connection=yes
Done.


SKP_CLIENT,NAME_CLIENT,AGE_SEGMENTATION
2745335,LE,36-50
502422,MINH,24-35
